In [ ]:
import cv2
import pandas as pd
import numpy as np

from keras.preprocessing.image import img_to_array, load_img
from matplotlib import pyplot as plt

#from keras.preprocessing.image import ImageDataGenerator

In [4]:
df_full = pd.read_csv("../data/preprocessed/df_mole_merged.csv")
df_full.sample(5)

,id,file_path,kat.Diagnose,maligne
2499,F575,../data/Mole_Data/SET_F/F575.BMP,1,False
1549,E600,../data/Mole_Data/SET_E/E600.BMP,1,False
794,D814,../data/Mole_Data/SET_D/D814.BMP,1,False
2685,F770,../data/Mole_Data/SET_F/F770.BMP,1,False
2287,F359,../data/Mole_Data/SET_F/F359.BMP,1,False


In [5]:
df_full.shape

(2900, 4)

## Split data by diagnostic

In [6]:
# filter_maligne = df_full["maligne"] == True

df_kat1 = df_full[df_full["kat.Diagnose"]==1]
df_kat2 = df_full[df_full["kat.Diagnose"]==2]
df_kat3 = df_full[df_full["kat.Diagnose"]==3]

print(df_kat1.shape)
print(df_kat2.shape)
print(df_kat3.shape)

(2271, 4)
(579, 4)
(50, 4)


In [7]:


def convert_image_to_array(files):
    images_as_array=[]
    for file in files:
        # Convert to Numpy Array
        images_as_array.append(img_to_array(load_img(file))/255)
    return np.array(images_as_array).astype("float32")


## Manage unbalanced data
We will increase the number of maligne record by using the ImageDataGenerator

In [8]:
# Load pictures in numpy array
img_kat1 = convert_image_to_array(df_kat1["file_path"])
print(img_kat1.shape)

img_kat2 = convert_image_to_array(df_kat2["file_path"])
print(img_kat2.shape)

img_kat3 = convert_image_to_array(df_kat3["file_path"])
print(img_kat3.shape)
print(type(img_kat3))

MemoryError: Unable to allocate 6.21 GiB for an array with shape (2271, 387, 632, 3) and data type float32

In [10]:
def generate_flipped_images(original_images):
    _, s1, s2, s3 = original_images.shape
    result = np.empty([0, s1, s2, s3], dtype=original_images.dtype)
    
    new_img_array=[]
    for img in original_images:
        # Horizontal flip
        flippedimage = cv2.flip(img, 1)
        new_img_array.append(flippedimage)
        # Vertical flip
        new_img_array.append(cv2.flip(img, 0))
        new_img_array.append(cv2.flip(flippedimage, 0))
    return np.array(new_img_array).astype("float32")



new_img_kat3 = generate_flipped_images(img_kat3)
print("images kat3 generated")
new_img_kat2 = generate_flipped_images(img_kat2)
print("images kat2 generated")

NameError: name 'img_kat3' is not defined

In [8]:
print(new_img_kat2.shape)
print(new_img_kat3.shape)

(1737, 387, 632, 3)
(150, 387, 632, 3)


In [9]:
target_kat1 = np.ones(img_kat1.shape[0]) # 111111
target_kat2 = np.ones(img_kat2.shape[0] + new_img_kat2.shape[0]) * 2 # 22222
target_kat3 = np.ones(img_kat3.shape[0] + new_img_kat3.shape[0]) * 3 # 33333

In [10]:
print(target_kat1.shape)
print(target_kat2.shape)
print(target_kat3.shape)

(2271,)
(2316,)
(200,)


In [21]:
final_features = np.concatenate((img_kat1, img_kat2, new_img_kat2, img_kat3, new_img_kat3))
final_target = np.concatenate((target_kat1, target_kat2, target_kat3))

print(final_features.shape)
print(final_target.shape)

(4787, 387, 632, 3)
(4787,)


In [22]:
np.save('./data/preprocessed/features.npy', final_features)
np.save('./data/preprocessed/target.npy', final_target)